In [4]:
import os
import numpy as np
import pandas as pd
import requests
from quantiprot.metrics import aaindex
from Bio import ExPASy, SwissProt, AlignIO, pairwise2
from Bio.PDB import *
from Bio.PDB.DSSP import DSSP, dssp_dict_from_pdb_file
from scipy.stats import entropy
from Bio.SeqUtils import seq1

path = '../'

d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
    'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
    'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
    'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M',
    'MSE': 'Z', 'UNK': '9', 'X': 'X'} 

def get_column_completeness(filename, column):
    """
    Derived using AliStats: https://github.com/thomaskf/AliStat
    Using (for each MSA): alistat $msa_name.a3m 6 -t 1,2,3 although
    technically only -t 2 (Table_2.csv) is needed for this function
    """
    df = pd.read_csv(filename)
    return df.at[column, 'Cc']


def get_alignment_summary(filename):
    """
    Derived using AliStats: https://github.com/thomaskf/AliStat
    Using (for each MSA): alistat $msa_name.a3m 6 -t 1,2,3 from Summary.txt
    """
    with open(filename, 'r') as f:
        for i, line in enumerate(f.readlines()):
            if i == 3:
                n_seqs = int(line[64:].strip())
            elif i == 6:
                completeness = float(line[64:].strip())
    return completeness, n_seqs


def get_conservation(alignment_file, residue_index, wild_type):
    """
    Obtains the entropy and (related) percent identity at a given position in
    the provided MSA
    """
    alignment = AlignIO.read(alignment_file, 'fasta')

    # obtain only the relevant column
    residues = [seq[residue_index] for seq in alignment]
    # the first sequence at this position is necessarily the wild-type
    assert residues[0] == wild_type, print('Unexpected residue in alignment')

    # compute the entropy based on the amounts of each residue type
    count = {aa: residues.count(aa) for aa in set(residues)}
    probability = [count[aa]/len(residues) for aa in count]
    entropy_score = entropy(probability)

    # compute percent residue identity, a.k.a. percent conservation of wild-type
    pri_score = count[wild_type] / len(residues) * 100
    return entropy_score, pri_score


def get_residue_features(uniprot_id, chain_id, residue_id):
    """
    Uses UniProt ID to get residue features from SwissProt
    Returns a list of features
    """
    handle = ExPASy.get_sprot_raw(uniprot_id)
    record = SwissProt.read(handle)
    features = []
    # iterate over all features for the given UniProt ID
    for feature in record.features:
        site_start = feature.location.start
        site_end = feature.location.end
        try:
            # only use this feature if the residue_id occurs within its range
            if residue_id >= site_start and residue_id <= site_end and \
                feature.type not in ['CHAIN', 'DOMAIN', 'REGION']: 
                features.append(feature.type)
        except Exception as e:
            print('Exception in get_residue_features:', e)
    return features


def get_interface_residues(model, target_chain_id, distance_threshold=10.0):
    """
    Determine which residues of a given chain are part of an interface based on
    a distance threshold between CAs of each residue on a target chain and any 
    residue on another chain
    """

    # Get the target chain and the list of other chains
    target_chain = None
    other_chains = []
    for chain in model.get_chains():
        if chain.get_id() == target_chain_id:
            target_chain = chain
        else:
            other_chains.append(chain)

    # Check if the target chain exists
    if target_chain is None:
        raise ValueError(f"Chain {target_chain_id} not found")

    # Store the interface residues of the target chain
    interface_residues = []
    # Iterate over all residues, (other) chains, and other chains' residues
    for other_chain in other_chains:
        for target_residue in target_chain.get_residues():
            for other_residue in other_chain.get_residues():
                try:
                    target_atom = target_residue['CA']
                    other_atom = other_residue['CA']
                except KeyError:
                    continue
                # distance determination
                distance = np.linalg.norm(
                    np.array(target_atom.coord) - np.array(other_atom.coord)
                    )
                # only count as interface if within the threshold
                if distance < distance_threshold:
                    interface_residues.append(target_residue.get_id()[1])
                    break
    return interface_residues


def get_residue_accessibility(filename, target_chain):
    """
    Run DSSP to determine the absolute surface area of each residue
    """
    dssp_dict = dssp_dict_from_pdb_file(filename)
    # get the target chain only
    df = pd.DataFrame(dssp_dict[0]).T.loc[target_chain, :]
    df.index = pd.Series(df.index).apply(lambda x: x[1])
    df = df.rename({0:'wild_type'}, axis=1)
    return df


def get_packing_density(model):
    """Not used (does not work)"""
    hse = HSExposureCA(model)
    asa_sum = sum(hse.get_exposed_total_rel())
    volume = hse.get_volume()
    print(volume)
    packing_density = asa_sum / volume
    return packing_density, asa_sum, volume


def get_h_angle(c_atom_coord, o_atom_coord, n_atom_coord):
    """
    Used in the determination of hydrogen bonds
    """
    a = np.array(c_atom_coord)
    b = np.array(o_atom_coord)
    c = np.array(n_atom_coord)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)


def get_residue_interactions(model, target_chain_id, residue_id):
    """
    Uses previous functions and internal logic to determine if a target residue
    is involved in hydrogen bonding or salt bridges (and also extracts the beta-
    factor)
    """

    # get the target chain object
    for chain in model.get_chains():
        if chain.get_id() == target_chain_id:
            target_chain = chain

    # get the target residue object
    target_residue = None
    for residue in target_chain.get_residues():
        if residue.get_id()[1] == residue_id:
            target_residue = residue
    if target_residue is None:
        return None, None, None, None
    
    h_bonds = 0
    salt_bridges = 0
    b_factor = []
    h_bond_identities = []

    # iterate through all atoms of the target residue
    for atom in target_residue.get_atoms():
        # extract beta factor
        b_factor.append(atom.get_bfactor())
        try:
            # detect all atom neighbors within 3.5 Angstrom
            neighbors = NeighborSearch(Selection.unfold_entities(
                model[target_chain_id], 'A'
                ), bucket_size=10).search(atom.coord, radius=3.5, level='A')
        except Exception as e:
            print('Exception in NeighborSearch,', e)
            continue
        # iterate through neighboring atoms within 3.5 Angstrom
        for neighbor in neighbors:
            # exclude niegbors from the same residue
            if neighbor.parent != atom.parent:
                # calculate the distance between the two
                dist = np.linalg.norm(atom.coord - neighbor.coord)
                try:
                    # hydrogen bonding logic: based on C/O/N adjacency
                    # hydrogens are implicit
                    h_angle = 0
                    if 'N' in atom.name and 'O' in neighbor.name:
                        # we want to find the C attached to O in neighbor
                        # so look for covalently bonded atoms
                        local = NeighborSearch(
                            list(neighbor.get_parent().get_atoms())
                            ).search(neighbor.coord, radius=1.5, level='A')
                        for l in local:
                            if 'C' in l.name:
                                target_atom = l
                        if target_atom is None:
                            print('did not find a closeby atom')
                            continue
                        # use the previous function to determine the angle
                        # and ultimately whether this matches h-bond constraints
                        h_angle = get_h_angle(
                            target_atom.coord, neighbor.coord, atom.coord
                            )
                    # same as above but with ids swapped
                    elif 'O' in atom.name and 'N' in neighbor.name:
                        # we want to find the C attached to O in atom
                        local = NeighborSearch(
                            list(atom.get_parent().get_atoms())
                            ).search(atom.coord, radius=1.5, level='A')
                        target_atom = None
                        for l in local:
                            if 'C' in l.name:
                                target_atom = l
                        if target_atom is None:
                            print('did not find a closeby atom')
                            continue
                        h_angle = get_h_angle(target_atom.coord, atom.coord, neighbor.coord)
                    # case where we have a sidechain O (e.g. OD, OE)
                    elif 'O' in atom.name and 'O' in neighbor.name and \
                        (atom.name != 'O' or neighbor.name != 'O'):
                        # neighbor is sidechain
                        if atom.name == 'O':
                            h_angle = get_h_angle(
                                atom.get_parent()['C'].coord, 
                                atom.coord, neighbor.coord
                                )
                        # atom (target) is sidechain O
                        elif neighbor.name == 'O':
                            h_angle = get_h_angle(
                                neighbor.get_parent()['C'].coord, 
                                atom.coord, neighbor.coord
                                )
                        # both are sidechains
                        else:
                            local = NeighborSearch(
                                list(neighbor.get_parent().get_atoms())
                                ).search(atom.coord, radius=1.5, level='A')
                            target_atom = None
                            for l in local:
                                if 'C' in l.name:
                                    target_atom = l
                            if target_atom is None:
                                print('did not find a closeby atom')
                                continue
                            h_angle = get_h_angle(target_atom.coord, 
                                atom.coord, neighbor.coord
                                )
                    else:
                        continue
                    # criteria for hydrogen bonding
                    if 120 < h_angle < 180:
                        h_bonds += 1
                        # for debugging
                        h_bond_identities.append((
                            f'{atom.name}-{atom.get_parent().id[1]}-H',
                            f'-{neighbor.name}-{neighbor.get_parent().id[1]}'
                        ))

                    # salt-bridge criteria
                    if atom.name in ['NH1', 'NH2', 'NZ'] and \
                        atom.parent.resname in ['LYS', 'ARG'] and \
                            neighbor.name in ['OE1', 'OE2', 'OD1', 'OD2'] and \
                                neighbor.parent.resname in ['ASP', 'GLU']:
                        salt_bridges += 1
                    elif neighbor.name in ['NH1', 'NH2', 'NZ'] and \
                        neighbor.parent.resname in ['LYS', 'ARG'] and \
                            atom.name in ['OE1', 'OE2', 'OD1', 'OD2'] and \
                                atom.parent.resname in ['ASP', 'GLU']:
                        salt_bridges += 1
                except Exception as e:
                    print('Exception inside residue interactions,', e)

    # take average over residue
    b_factor = sum(b_factor) / len(b_factor)
        
    return h_bonds, salt_bridges, b_factor, h_bond_identities


def extract_features(database_loc, path='~/OneDrive/thermostability_transfer'):
    """
    Generate all the features and add them to the dataset_mapped.csv
    """

    # list for storing DSSP outputs
    dfs_acc = []

    # construct the output dataframe
    df_out = pd.DataFrame(index=db.index, 
        columns=['on_interface', 'entropy', 'conservation', 
                 'column_completeness', 'completeness_score', 'n_seqs', 
                 'structure_length', 'features', 'hbonds', 'h_bond_ids', 
                 'saltbrs', 'b_factor', 'residue_depth'])

    df_out['code'] = df_out.index.str[:4]
    df_out['on_interface'] = False

    # sequence-based features from QuantiProt
    vol = aaindex.get_aa2volume() # residue total volume
    kdh = aaindex.get_aa2hydropathy() # Kyte-Doolittle hydrophobicity
    chg = aaindex.get_aa2charge() # neutral-pH charge

    # get the first instance of a unique mutation only (avoid duplications)
    db = pd.read_csv(database_loc)
    if 'fireprot' in database_loc.lower():
        dataset = 'fireprot'
    elif 's669' in database_loc.lower():
        dataset = 's669'
    db = db.groupby('uid').first()
    db['offset_rosetta'] = db['offset_rosetta'].fillna(0)

    # compute the sequence-based features
    df_out['kdh_wt'] = db['wild_type'].apply(lambda x: kdh[x])
    df_out['kdh_mut'] = db['mutation'].apply(lambda x: kdh[x])
    df_out['vol_wt'] = db['wild_type'].apply(lambda x: vol[x])
    df_out['vol_mut'] = db['mutation'].apply(lambda x: vol[x])
    df_out['chg_wt'] = db['wild_type'].apply(lambda x: chg[x])
    df_out['chg_mut'] = db['mutation'].apply(lambda x: chg[x])
     
    # iterate through each unique wild-type PDB structure
    for code, group in tqdm(db.groupby('code')):
        print(code)

        # get the structure and target chain where the mutation is
        pdb_file = group['pdb_file'].head(1).item()
        target_chain_id = group['chain'].head(1).item()

        # parse the structure and get its high-level model object
        structure = PDBParser().get_structure('PDB_ID', pdb_file)
        model = structure[0]

        # according to Bio.PDB
        rd = ResidueDepth(model)

        # create a mapping of chain to tuples of (resnum, resname)
        chains = {chain.id:
                    [(residue.id[1], residue.resname) for residue in chain] \
                  for chain in structure.get_chains()
                 }

        # get interface residues less than 7 Angstrom from another chain
        interface_residues = get_interface_residues(
            model, target_chain_id, distance_threshold=7
            )
        
        # get the sequence of the chain of interest
        pdb_seq = chains[target_chain_id]
        # indices of this sequence (1...N)
        indices = [val[0] for val in pdb_seq]
        # 1 letter code for residues in the sequence
        aas = [d[val[1]] for val in pdb_seq]

        # convert to 1-based indexing
        interface_residues_indices = [
            indices.index(v)+1 for v in interface_residues
            ]
        # indexing a list which is 0-based using 1-based indexing
        interface_residue_identities = [
            aas[v-1] for v in interface_residues_indices
            ]

        try:
            # according to DSSP
            df_acc = get_residue_accessibility(
                pdb_file, target_chain=group['chain'].head(1).item()
                )
            # add information so that this dataframe can be joined later
            df_acc['code'] = code
            df_acc = df_acc.reset_index()
            # convert to 1-based
            df_acc['position'] = df_acc['index'].apply(
                lambda x: indices.index(x)+1
                )
            dfs_acc.append(df_acc)
        except Exception as e:
            print('Exception inside residue accessibility,', e)
        
        # now iterate through each unique mutation
        for uid, row in group.iterrows():
            df_out.at[uid, 'structure_length'] = len(pdb_seq)

            wt = row['wild_type']
            target_pos = row['position'] + \
                row['offset_up'] * (0 if dataset == 's669' else 1)

            res0 = Selection.unfold_entities(model[target_chain_id], 'R')[0]
            # should always start as 1 following recent changes
            offset = res0.id[1]
            # handle unknowns not being parsed
            if res0.resname == 'UNK':
                offset += 1
            # corrected position
            target_pos += offset - 1

            try:
                #validation
                assert d[model[target_chain_id][target_pos].resname] == wt
                # only assign validated residues
                df_out.at[uid, 'residue_depth'] = rd[
                    target_chain_id, (' ', target_pos, ' ')][0]
            except:
                try:
                    # indicate what discrepancy occured
                    print('wt:', wt, 'obs', 
                        d[model[target_chain_id][target_pos].resname], 
                        'target_pos', target_pos)
                    print([e.get_name() for e in Selection.unfold_entities(
                            model[target_chain_id], 'R'
                            )])
                except:
                    print('Could not find', target_pos)

            # next section uses sequence-features, which are indexed differently

            # -1 for 0-based indexing
            target_pos_up = row['position'] + \
                row['offset_up'] * (-1 if dataset == 's669' else 0) - 1
            alignment_file = row['msa_filename']
            
            try:
                entropy, pri = get_conservation(
                    alignment_file, target_pos_up, wild_type=row['wild_type']
                    )
                df_out.at[uid, 'entropy'] = entropy
                df_out.at[uid, 'conservation'] = pri

                # assumes predictions and msa stats from AliStats are saved
                # in folders within predictions directory
                df_out.at[uid, 'column_completeness'] = get_column_completeness(
                        os.path.join(path, 'predictions', 
                        code+'_'+row['chain'], 'Table_2.csv'
                        ), target_pos_up + 1)
                
                completeness, n_seqs = get_alignment_summary(
                    os.path.join(path 'predictions', 
                    code+'_'+row['chain'], 'Summary.txt'))
                #print(completeness, n_seqs, 1)
                df_out.at[uid, 'completeness_score'] = completeness
                df_out.at[uid, 'n_seqs'] = n_seqs

            except Exception as e:
                print('Exception inside alignment', e)

            if target_pos in interface_residues_indices:
                # validate again
                assert wt == interface_residue_identities[
                    interface_residues_indices.index(target_pos)]
                df_out.at[uid, 'on_interface'] = True
                df_out.at[uid, 'target_position'] = target_pos

                #except:
                #        print(wt, target_pos_0)
                #        print(interface_residues_indices)
                #        print(interface_residue_identities)

            hbonds, saltbrs, b_factor, h_bond_identities = \
                get_residue_interactions(model, target_chain_id, target_pos)
            df_out.at[uid, 'hbonds'] = hbonds
            df_out.at[uid, 'h_bond_ids'] = h_bond_identities
            df_out.at[uid, 'saltbrs'] = saltbrs
            df_out.at[uid, 'b_factor'] = b_factor

            if row['uniprot_id'] is not None:
                # undo zero-based indexing
                try:
                    df_out.at[uid, 'features'] = get_residue_features(
                        row['uniprot_id'], None, target_pos_up + 1
                        )
                except Exception:
                    print(uid, 'Timed out')
                    df_out.at[uid, 'features'] = 'TIMEOUT'
            else:
                df_out.at[uid, 'features'] = None

    return df_out.drop('code', axis=1), pd.concat(dfs_acc)


if __name__=='__main__':
    parser = argparse.ArgumentParser(
                    description = 'Processes features from either s669 or \
                                   FireProtDB for downstream analysis'
                    )
    parser.add_argument('--db_loc', help='location of the database csv file.' 
                      +' Must contain the name of the database (s669/fireprot)')
    parser.add_argument('-o', '--output_root', 
                        help='root of folder to store outputs')

    args = parser.parse_args()

    db = pd.read_csv(args.db_loc)
    db = db.groupby('uid').first()
    db['offset_rosetta'] = 0

    feat, dssp = extract_features(
        args.db_loc, args.output_root)

    feat_2 = db.join(feat, how='left')

    dssp.loc[dssp['wild_type'].isin(['a','b']), 'wild_type'] = 'C'
    out = feat_2.merge(dssp[['code', 'wild_type', 1, 2, 'position']],
        on=['code', 'wild_type', 'position'], how='left').rename(
            {2: 'rel_ASA', 1: 'SS'}, axis=1)
    out.to_csv(args.db_loc.replace('_mapped.csv', '_mapped_feats.csv')) 

In [13]:
# is the right chain being references
db = pd.read_csv(os.path.join(path, 's669_mapped.csv'))
db = db.groupby('uid').first()
db['offset_rosetta'] = 0

feat_s669, dssp_s669 = extract_features(db, dataset='s669', path=path)
feat_s669_2 = db.join(feat_s669, how='left')
dssp_s669.loc[dssp_s669['wild_type'].isin(['a','b']), 'wild_type'] = 'C'
out_s669 = feat_s669_2.merge(dssp_s669[['code', 'wild_type', 1, 2, 'position']], on=['code', 'wild_type', 'position'], how='left').rename({2: 'rel_ASA', 1: 'SS'}, axis=1)
out_s669.to_csv(os.path.join(path, 's669_mapped_feats.csv'))

/tmp/ipykernel_4711/2103696169.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for code, group in tqdm(db.groupby('code')):


  0%|          | 0/94 [00:00<?, ?it/s]

1A0F
1A7V
1BA3
1BFM
1BNL
Exception in get_residue_features, '>=' not supported between instances of 'int' and 'UnknownPosition'
1D5G
1DIV
did not find a closeby atom
1DXX
Exception inside alignment No records found in handle
1EKG
did not find a closeby atom
1F8I
1FH5
1FRD
1FT8
1FXA
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
1G3P
1GLU
1GUA
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
1GWY
1H0X
1HCQ
1IOJ
1IR3
1ITM
1IV7
Exception inside alignment index out of range
Exception inside alignment index out of range
Exception inside alignment index out of range
1IV9
Exception inside alignment index out of range
1J8I
1JL9
Exception inside alignment No records found in handle
1JLV
1L6H
did not find a closeby atom
did not find a closeby atom
1LVM
1N18
1N88
1NM1
1O1U
did not find a closeby atom
did not find a closeby atom
1O6X
did not find a closeby atom
1OSI
1PFL
did not find a closeby atom


tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0


did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
3ECU
3FIS
3G1G
3K82
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
3L15
3MON
3O39
did not find a closeby atom
3S4M
3S92
4BJX
4BUQ
did not find a closeby atom
4HE7
4N6V
4WAA
4YEE
4YEF
5JXB
5OAQ
5VP3


In [49]:
print(dssp_fp.loc[dssp_fp['wild_type'].isin(['a','b']), 'wild_type'])
dssp_fp.loc[dssp_fp['wild_type'].isin(['a','b']), 'wild_type'] = 'C'

db = db.rename({'conservation': 'conservation_fp', 'b_factor': 'b_factor_fp'}, axis=1)
feat_fp = db.join(feat_fp_raw, how='left')
feat_fp['position'] = feat_fp['position'] + feat_fp['offset_up']
feat_fp = feat_fp.merge(dssp_fp[['code', 'wild_type', 1, 2, 'position']], on=['code', 'wild_type', 'position'], how='left').rename({2: 'rel_ASA', 1: 'SS'}, axis=1)
feat_fp['position'] = feat_fp['position'] - feat_fp['offset_up']
feat_fp.to_csv(os.path.join(path, 'fireprot_mapped_feats.csv'))

Series([], Name: wild_type, dtype: object)


In [16]:
db = pd.read_csv(os.path.join(path, 'fireprot_mapped.csv'))
db = db.groupby('uid').first()
### HACK
db['offset_rosetta'] = 0

feat_fp_raw, dssp_fp = extract_features(db, dataset='fireprot', path=path)
dssp_fp.loc[dssp_fp['wild_type'].isin(['a','b']), 'wild_type'] = 'C'

db = db.rename({'conservation': 'conservation_fp', 'b_factor': 'b_factor_fp'}, axis=1)
feat_fp = db.join(feat_fp_raw, how='left')
feat_fp['position'] = feat_fp['position'] + feat_fp['offset_up']
feat_fp = feat_fp.merge(dssp_fp[['code', 'wild_type', 1, 2, 'position']], on=['code', 'wild_type', 'position'], how='left').rename({2: 'rel_ASA', 1: 'SS'}, axis=1)
feat_fp['position'] = feat_fp['position'] - feat_fp['offset_up']
feat_fp.to_csv(os.path.join(path, 'fireprot_mapped_feats.csv'))
#feat_fp.to_csv(os.path.join('/home/sareeves/OneDrive/fireprot_mapped_feats.csv'))

/tmp/ipykernel_4711/2103696169.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for code, group in tqdm(db.groupby('code')):


  0%|          | 0/230 [00:00<?, ?it/s]

1A23
1A43
1A5E
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1A5E_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1A5E_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1A5E_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1A5E_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1A5E_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1A5E_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1A5E_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1A5E_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file

sphere_mange_arete: inconcistence


did not find a closeby atom
1CQW
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1CQW_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1CQW_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1CQW_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1CQW_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1CQW_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1CQW_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1CQW_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1CQW_A/Table_2.csv'
Exception inside alignment [Err

tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0
tri_norm: face with normal vector of lenght 0


did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
did not find a closeby atom
1POH
did not find a closeby atom
1PX0
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1PX0_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1PX0_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1PX0_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1PX0_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1PX0_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1PX0_A/Table_2.csv'
Exception inside alignment [Errno 2] No such file or directory: '/home/sareeves/fast/predictions/1PX0_A/Table_2.csv'
Exc

In [13]:
feat_fp.loc[feat_fp['on_interface']].head()

,Unnamed: 0.1,experiment_id,protein_name,uniprot_id,pdb_id,ddG,dTm,is_curated,type,derived_type,...,on_interface,entropy,conservation,kdh_wt,kdh_mut,vol_wt,vol_mut,chg_wt,chg_mut,target_position
uid,,,,,,,,,,,,,,,,,,,,,
1AAR_680Q,33409,PT005987,Polyubiquitin-C,P0CH28,1AAR|1AAR,0.33,NaN,True,NaN,NaN,...,True,0.636514,33.333333,-4.5,-3.5,6.13,3.95,1,0,72.0
1ADO_129A,43402,PT010969,Fructose-bisphosphate aldolase A,P00883,1ADO|1ADO|1ADO|1ADO,NaN,-10.7,True,NaN,NaN,...,True,0.338584,93.904168,-3.5,1.8,2.78,1.00,-1,0,128.0
1ADO_129G,43370,PT010968,Fructose-bisphosphate aldolase A,P00883,1ADO|1ADO|1ADO|1ADO,NaN,-11.7,True,NaN,NaN,...,True,0.338584,93.904168,-3.5,-0.4,2.78,0.00,-1,0,128.0
1ADO_129N,43434,PT010970,Fructose-bisphosphate aldolase A,P00883,1ADO|1ADO|1ADO|1ADO,NaN,-10.1,True,NaN,NaN,...,True,0.338584,93.904168,-3.5,-3.5,2.78,2.95,-1,0,128.0
1ADO_129Q,43466,PT010971,Fructose-bisphosphate aldolase A,P00883,1ADO|1ADO|1ADO|1ADO,NaN,-9.8,True,NaN,NaN,...,True,0.338584,93.904168,-3.5,-3.5,2.78,3.95,-1,0,128.0


In [53]:
from Bio import AlignIO
from scipy.stats import entropy

def get_conservation(alignment, residue_index, wild_type):
    residues = [seq[residue_index] for seq in alignment]
    count = {aa: residues.count(aa) for aa in set(residues)}
    probability = [count[aa]/len(residues) for aa in count]
    entropy_score = entropy(probability)
    pri_score = count[wild_type] / len(residues) * 100
    return (entropy_score, pri_score)

# Example usage
alignment = AlignIO.read('/root/fast/predictions/1A0F_A/1A0F_MSA_full_cov75_id90_reduced.a3m', 'fasta')
residue_index = 30
conservation_score = get_conservation_score(alignment, residue_index)
print(f"Conservation score of residue at index {residue_index}: {conservation_score}")

Conservation score of residue at index 30: 1.708321187742682


In [31]:
feat_s669[feat_s669['on_interface'], ['wild_type', 'wt_id', 'position', 'target_position', 'mutation', 'multimer', 'on_interface', 'kdh_wt', 'kdh_mut', 'vol_wt', 'vol_mut']]

InvalidIndexError: (uid
1A0F_11A     False
1A7V_104H    False
1A7V_13H     False
1A7V_20H     False
1A7V_31H     False
             ...  
5JXB_25P     False
5OAQ_199H    False
5VP3_128G    False
5VP3_183T    False
5VP3_39K     False
Name: on_interface, Length: 669, dtype: bool, ['wild_type', 'wt_id', 'position', 'target_position', 'mutation', 'multimer', 'on_interface', 'kdh_wt', 'kdh_mut', 'vol_wt', 'vol_mut'])

In [134]:
code='1ACB'
entity=2
pos=72
#req = f'https://www.ebi.ac.uk/pdbe/graph-api/mappings/ec/{code.lower()}'
req = f'https://www.ebi.ac.uk/pdbe/graph-api/pdbe_pages/interfaces/{code.lower()}/{entity}'
r = requests.get(req).text.replace('true','True').replace('false','False')
r = eval(r)
r
#pd.DataFrame(r[code.lower()]['data'][0]['residues'])

{'1acb': {'sequence': 'TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNHVPHVG',
  'length': 70,
  'dataType': 'INTERFACE RESIDUES',
  'data': [{'dataType': 'UniProt',
    'accession': 'P00766',
    'name': 'Chymotrypsinogen A',
    'additionalData': {'bestChainId': 'B', 'entityId': 2},
    'residues': [{'startIndex': 43,
      'endIndex': 43,
      'startCode': 'VAL',
      'endCode': 'VAL',
      'indexType': 'PDB'},
     {'startIndex': 45,
      'endIndex': 48,
      'startCode': 'LEU',
      'endCode': 'ARG',
      'indexType': 'PDB'}]}]}}

In [ ]:
         # uniprot entries corresponding to multichain PDBs sometimes need to be specified   
          if code in ['1ACB', '1AON', '1GUA', '1GLU', '1JIW', '2CLR', '3MON']:
               entity = 2
          elif code in ['1HCQ', '1TUP', '3DV0']:
               entity = 3
          else:
               entity = 1
               
          req = f'https://www.ebi.ac.uk/pdbe/graph-api/mappings/ec/{code.lower()}'
          r = requests.get(req).text.replace('true','True').replace('false','False').replace('null','None')
          r = eval(r)
          try:
               ec = list(r[code.lower()]['EC'].keys())[0]
               df_out.loc[df_out['code']==code, 'EC'] = ec
          except:
               pass
          
          req = f'https://www.ebi.ac.uk/pdbe/graph-api/pdbe_pages/interfaces/{code.lower()}/{entity}'
          r = requests.get(req).text.replace('true','True').replace('false','False')
          r = eval(r)
          if r == {} and group['multimer'].head(1).item() > 1:
               print(code, 'missing interface')
          try:
               df = pd.DataFrame(r[code.lower()]['data'][0]['residues'])
               db_seq = r[code.lower()]['sequence']
               orig_seq = group['pdb_ungapped'].head(1).item()
               alignments = pairwise2.align.globalms(db_seq, orig_seq, 3, -1, -1, -0.1) #match, miss, open, extend
               choice = alignments[0]
               gapped = choice.seqB
               if '-' in gapped.strip('-'):
                    print(code, gapped)
          except:
               df = pd.DataFrame({'startIndex': [1], 'endIndex': [0], 'indexType': 'PDB'})
               gapped = None
          assert (df['indexType'] == 'PDB').all()

          for uid, row in group.iterrows():  
               target_pos = row['position'] + row['offset_up'] * (0 if dataset == 's669' else 1)
               if gapped is not None:
                    i = 0
                    offset_target_pos = 0
                    while i <= target_pos:
                         if gapped[i] == '-':
                              offset_target_pos += 1
                         i += 1
                    target_pos += offset_target_pos
               elif row['multimer'] > 1:
                    df_out.at[uid, 'is_interface'] = None
               else:
                    df_out.at[uid, 'is_interface'] = False
               pos = target_pos

               for _, row2 in df.iterrows():
                    if pos >= row2['startIndex'] and pos <= row2['endIndex']:
                         try:
                              if pos == row2['startIndex']:
                                   assert row['wild_type'] == d[row2['startCode']], (code, pos, row['position'], row['wild_type'], d[row2['startCode']])
                              elif pos == row2['endIndex']:
                                   assert row['wild_type'] == d[row2['endCode']], (code, pos, row['position'], row['wild_type'], d[row2['startCode']])
                              df_out.at[uid, 'is_interface'] = True
                              df_out.at[uid, 'offset_new'] = gapped[:pos].count('-')
                         except AssertionError as e:
                              print('AssertionError')
                              print(e)
                         except KeyError as e:
                              print('KeyError')
                              print(code)
                              print(e)  

     return df_out.drop('code', axis=1)   